In [ ]:
!pip install pyngrok

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

In [ ]:
!apt install nvidia-cuda-toolkit -y

In [ ]:
!make server -C /kaggle/working/llama.cpp LLAMA_CUDA=1

In [ ]:
!huggingface-cli download Qwen/CodeQwen1.5-7B-Chat-GGUF codeqwen-1_5-7b-chat-q4_k_m.gguf --local-dir . --local-dir-use-symlinks False
!mv /kaggle/working/codeqwen-1_5-7b-chat-q4_k_m.gguf /kaggle/working/llama.cpp/models

In [ ]:
# ngrokToken留空则使用localtunnel进行内网穿透
ngrokToken = "2g5SoIqUMNlj08BlPCgAdQDA7cy_3wUF54dZjMfAJRgh4Mjgu"
use_pinggy = True

from huggingface_hub import hf_hub_download
from pathlib import Path

if ngrokToken:
    from pyngrok import conf, ngrok
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False
    ssh_tunnels = ngrok.get_tunnels(conf.get_default())
    if len(ssh_tunnels) == 0:
        ssh_tunnel = ngrok.connect(8080)
        print('address：'+ssh_tunnel.public_url)
    else:
        print('address：'+ssh_tunnels[0].public_url)
elif use_pinggy:
    import subprocess
    import threading
    def start_pinggy(port):
        cmd = f"ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -p 443 -R0:localhost:{port} a.pinggy.io"
        p = subprocess.Popen(cmd.split(" "), stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line.decode(), end='')
    threading.Thread(target=start_pinggy, daemon=True, args=(8080,)).start()
else:
    import subprocess
    import threading
    def start_localtunnel(port):
        p = subprocess.Popen(["lt", "--port", f"{port}"], stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line.decode(), end='')
    threading.Thread(target=start_localtunnel, daemon=True, args=(8080,)).start()



!/kaggle/working/llama.cpp/server -m /kaggle/working/llama.cpp/models/codeqwen-1_5-7b-chat-q4_k_m.gguf -c 2048 -ngl 10000

address：https://c7bc-34-80-242-244.ngrok-free.app
{"tid":"139557030264832","timestamp":1716002129,"level":"INFO","function":"main","line":2942,"msg":"build info","build":2918,"commit":"05834841"}
{"tid":"139557030264832","timestamp":1716002129,"level":"INFO","function":"main","line":2947,"msg":"system info","n_threads":2,"n_threads_batch":-1,"total_threads":4,"system_info":"AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | "}
llama_model_loader: loaded meta data with 23 key-value pairs and 387 tensors from /kaggle/working/llama.cpp/models/codeqwen-1_5-7b-chat-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_mode